In [1]:
import numpy as np
import tensorflow as tf 
from keras.models import load_model

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,Subtract,Reshape
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D,Conv2D,MaxPooling2D,Input,Lambda,GlobalMaxPooling2D
from keras.regularizers import l2
from keras import backend as K
from keras.applications.vgg16 import VGG16
from skimage.io import imsave

from matplotlib.pyplot import imread
from skimage.transform import rescale, resize
import os

In [2]:
# Data 
reshape_size = [224,224,3]
dataset_path = '../../datasets/VisualPhish/'
data_path = dataset_path + 'newly_crawled_phishing/'
imgs_num = 1089#955

# Saved models and computed embeddings
margin = 2.2
#output_dir = './'
output_dir = '../train/'
model_name = '../train/model.h5'

phish_emb_name = 'phishing_emb.npy'
phish_emb_labels_name = 'phishing_labels.npy'

phish_train_idx_name = 'train_idx.npy'
phish_test_idx_name = 'test_idx.npy'

train_emb_name = 'whitelist_emb.npy'
train_emb_labels_name = 'whitelist_labels.npy'

# Load New phishing examples

In [3]:
def read_new_phish_imgs(data_path,targets,imgs_num,reshape_size,labels_dict):
    all_imgs = np.zeros(shape=[imgs_num,224,224,3])
    all_labels = np.zeros(shape=[imgs_num,1])
    all_file_names = []
    label_names = sorted(os.listdir(data_path))
    count = 0
    #print(targets)
    for i in range(0,len(targets)):
        #for j in range(0,imgs_num):
        label=targets[i]
        target_path = data_path + targets[i]
        file_names = sorted(os.listdir(target_path))
        for j in range(0,len(file_names)):
            if not file_names[j] == 'labels.txt':
                print(file_names[j])
                img = imread(os.path.join(target_path,file_names[j]))
                img = img[:,:,0:3]
                all_imgs[count,:,:,:] = resize(img, (reshape_size[0], reshape_size[1]),anti_aliasing=True)
                #print(label)
                #print(labels_dict)
                all_labels[count,:] = labels_dict[label]
                all_file_names.append(file_names[j])
                count = count + 1
    print(all_imgs.shape)
    return all_imgs,all_labels,all_file_names

#get names of labels/indices
def get_dict_targets(data_path,file_name):
    targets_file = open(data_path+file_name, "r")
    targets = targets_file.read()
    targets_list = targets.splitlines()
    counter=0
    x = {'absa':0}
    for i in range(0,len(targets_list)):
        label=targets_list[i]
        target_path = data_path + targets_list[i]
        file_names = sorted(os.listdir(target_path))
        for j in range(0,len(file_names)):
            if not( file_names[j] == 'textfiles' or file_names[j] == 'textfiles_translated' ): 
                first_file = file_names[j]
                first_file = file_names[j].replace('T','')
                #label = int(first_file.split('_')[0])
                #d = {targets_list[i]:label}
                #x.update(d)
                if(label not in x):
                    counter+=1
                    x[label]=counter
                break
    return x
        
labels_dict = get_dict_targets(dataset_path+'phishing/','targets.txt')
print(labels_dict)

{'absa': 0, 'adobe.com': 1, 'github.com': 2, 'player.vimeo.com': 3, 'wordpress.com': 4, 'amazon.com': 5, 'google.com': 6, 'play.google.com': 7, 'wordpress.org': 8, 'apache.org': 9, 'gravatar.com': 10, 'qq.com': 11, 'yahoo.com': 12, 'apple.com': 13, 'itunes.apple.com': 14, 'support.google.com': 15, 'youtube.com': 16, 'blogspot.com': 17, 'linkedin.com': 18, 'tumblr.com': 19, 'en.wikipedia.org': 20, 'microsoft.com': 21, 'twitter.com': 22, 'flickr.com': 23, 'nytimes.com': 24, 'wikipedia.org': 25}


In [4]:
targets_file = open(data_path+'labels.txt', "r")
targets = targets_file.read()
targets = targets.splitlines()
all_imgs_phish_new,all_labels_phish_new,all_file_names_phish_new = read_new_phish_imgs(data_path,targets,imgs_num,reshape_size,labels_dict)

07b6d1973f.png
142a9f9316.png
14cfe4c792.png
21a5cfb8b1.png
25d554a03d.png
28a98eb5bb.png
29832390cb.png
37c10b1584.png
39a6fe9502.png
3c222291f1.png
3e621a121c.png
474c8be3ed.png
52e01d8271.png
53a1bc560b.png
5d259fce85.png
5e73e274d0.png
7469900ba8.png
7593975a3d.png
7b71059461.png
8fd218a07b.png
a3a7399428.png
a83010504e.png
aadb3d0ecb.png
b01bde5cc8.png
b06233d79e.png
b2179baf68.png
b457de7d15.png
b9e2b8f329.png
be521c8c4a.png
beb2dc461b.png
c48c8c1c3c.png
c6eeb2dfc4.png
c9fabd0c2e.png
cb4d866b65.png
cce0cae153.png
cd81b9b795.png
d154dfc7fc.png
d852585ec3.png
e0ae04d0b2.png
e43521db3c.png
ebd1b2f4ac.png
ed8f8a4968.png
f00f295f43.png
f052793e8e.png
f93f4b7d79.png
fd990f5de6.png
1184ba2307.png
19a3b8473d.png
1dc4fb73f3.png
1f80c6ab1c.png
22f9fbbe31.png
3097fca9b1.png
30e162bdc9.png
326044a620.png
3b79b0e141.png
3d6f61e0dd.png
45f4032fa9.png
4b83f9a8b2.png
4fe08ba2e0.png
50bfdf703c.png
52f5d1bbf7.png
59295a09b7.png
5beac8ac19.png
6d608df469.png
759c0450d2.png
79a7ddeed6.png
7b6de2862d

26096bbc1b.png
29db56248a.png
3181ac5b3f.png
3ba740817c.png
421fd7a13e.png
42b5d01b96.png
460ca08ad1.png
4c1d470718.png
4e66f79dab.png
4e8ceee157.png
53a439a685.png
6709154e91.png
6c32ebc6b5.png
6d816308fe.png
708e4c2ea1.png
754b3870a2.png
894739cce6.png
93010fd502.png
97f48f7ba8.png
a19fa662d9.png
b925bd7606.png
c3b86ee966.png
ca3e4ac819.png
cb32cc3527.png
df97747b96.png
e6fb2437aa.png
e8d6e0a078.png
f53d4f32e1.png
f71b1cd6d7.png
fbfcedde28.png
fc292b8ca3.png
fdbc41c3bc.png
08ec395ac8.png
09bbf37d06.png
1dc62d97a6.png
21d4a7ae35.png
25f835bffb.png
2615b8b335.png
2dd9442cc4.png
3ea43305e7.png
4352b24cd6.png
4b4773bdbc.png
4c325b4a0b.png
4f2dcc9609.png
534a25ef04.png
5373491ecd.png
53c1ecbb75.png
5c452b6ff1.png
69a76b3697.png
6c55e0a975.png
6d66c98480.png
702123450a.png
744a6b113a.png
83ef103b6d.png
846bca7ea7.png
85ad809693.png
8b9207b6dd.png
8c4fd912a3.png
91dc1cc0d1.png
97b44b3ce8.png
983bcf6c61.png
9a2b93c4cb.png
9aa2d99965.png
9b4e9a81d9.png
9f649c8d66.png
a5b5f56548.png
a933e898aa

# Load model and predict

In [5]:
# Load_model 
def loss(y_true,y_pred):
    loss_value = K.maximum(y_true, margin + y_pred)
    loss_value = K.mean(loss_value,axis=0)
    return loss_value

final_model = load_model(model_name, custom_objects={'loss': loss})
inside_model = final_model.layers[3]

X_phish_new = inside_model.predict(all_imgs_phish_new)

# Load precomputed embeddings 

In [6]:
X_legit_train = np.load(output_dir+train_emb_name)
y_legit_train = np.load(output_dir+train_emb_labels_name)

X_phish = np.load(output_dir+phish_emb_name)
y_phish = np.load(output_dir+phish_emb_labels_name)

phish_test_idx = np.load(output_dir+phish_test_idx_name)
phish_train_idx = np.load(output_dir+phish_train_idx_name)

X_phish_test = X_phish[phish_test_idx,:]
y_phish_test = y_phish[phish_test_idx,:]

X_phish_train = X_phish[phish_train_idx,:]
y_phish_train = y_phish[phish_train_idx,:]

label_dict={}#reverse lookup for file name, return labels
# Get file names of each example 
def read_file_names(data_path,file_name):
    targets_file = open(data_path+file_name, "r")
    targets = targets_file.read()
    
    file_names_list = []
    targets_list = targets.splitlines()
    for i in range(0,len(targets_list)):
        target_path = data_path + targets_list[i]
        #new add stuff
        label=targets_list[i]
        file_names = sorted(os.listdir(target_path))
        for j in range(0,len(file_names)):
            file_names_list.append(file_names[j])
            #new add stuff
            label_dict[file_names[j]]=label
    return file_names_list

legit_file_names = read_file_names(dataset_path+'trusted_list/','targets.txt')
phish_file_names = read_file_names(dataset_path+'phishing/','targets.txt')

phish_train_file_names = []
for i in range(0,phish_train_idx.shape[0]): 
    phish_train_file_names.append(phish_file_names[phish_train_idx[i]])
    
phish_test_file_names = []
for i in range(0,phish_test_idx.shape[0]):
    phish_test_file_names.append(phish_file_names[phish_test_idx[i]])

# def get_label_from_name(name):
#     first_half = name.split('_',1)[0]
#     number = int(first_half.replace('T',''))
#     return number
def get_label_from_name(name):
    label = label_dict[name]
    label_int = labels_dict[label]
    return label_int

# Compute distances

In [7]:
# L2 distance
def compute_distance_pair(layer1,layer2):
    diff = layer1 - layer2
    l2_diff = np.sum(diff**2) / X_phish_train.shape[1]
    return l2_diff

# Pairwise distance between query image and training
def compute_all_distances(test_matrix):
    train_size = X_phish_train.shape[0] + X_legit_train.shape[0]
    X_all_train = np.concatenate((X_phish_train,X_legit_train))
    pairwise_distance = np.zeros([test_matrix.shape[0],train_size])
    for i in range(0,test_matrix.shape[0]):
        pair1 = test_matrix[i,:]
        for j in range(0,train_size):
            pair2 = X_all_train[j,:]
            l2_diff = compute_distance_pair(pair1,pair2)
            pairwise_distance[i,j] = l2_diff
    return pairwise_distance

pairwise_distance = compute_all_distances(X_phish_new)

# Find Smallest n distances
def find_min_distances(distances,n):
    idx = distances.argsort()[:n]
    values = distances[idx]
    return idx,values

# Find names of examples with min distance
def find_names_min_distances(idx,values):
    names_min_distance = ''
    only_names = []
    distances = ''
    for i in range(0,idx.shape[0]):
        index_min_distance = idx[i]
        if (index_min_distance < X_phish_train.shape[0]):
            names_min_distance = names_min_distance + 'Phish: ' + phish_train_file_names[index_min_distance] +','
            only_names.append(phish_train_file_names[index_min_distance])   
        else:
            names_min_distance = names_min_distance + 'Legit: ' + legit_file_names[index_min_distance-X_phish_train.shape[0]] +','
            only_names.append(legit_file_names[index_min_distance-X_phish_train.shape[0]])   
        distances = distances + str(values[i]) + ','
    names_min_distance = names_min_distance[:-1]
    distances = distances[:-1]
    return names_min_distance,only_names,distances

# Find same-category website (matching is correct if it was matched to the same category (e.g. microsoft and outlook ))
parents_targets = ['microsoft','apple','google','alibaba']
sub_targets = [['ms_outlook','ms_office','ms_bing','ms_onedrive','ms_skype'],['itunes','icloud'],['google_drive'],['aliexpress']]

parents_targets_idx = [90,12,65,4]
sub_targets = [[150,152,151,149,148],[153,154],[147],[5]]

def check_if_same_category(img_label1,img_label2):
    if_same = 0
    if img_label1 in parents_targets_idx:
        if img_label2 in sub_targets[parents_targets_idx.index(img_label1)]:
            if_same = 1
    elif img_label1 in sub_targets[0]:
        if img_label2 in sub_targets[0] or img_label2 == parents_targets_idx[0]:
            if_same = 1
    elif img_label1 in sub_targets[1]:
        if img_label2 in sub_targets[1] or img_label2 == parents_targets_idx[1]:
            if_same = 1
    elif img_label1 in sub_targets[2]:
        if img_label2 in sub_targets[2] or img_label2 == parents_targets_idx[2]:
            if_same = 1
    return if_same

# Find if target is in the top closest n distances
def check_if_target_in_top(test_label,only_names):
    found = 0
    idx = 0
    #test_label = get_label_from_name(test_file_name)
    print('***')
    print('Target: '+str(test_label))
    for i in range(0,len(only_names)):
        label_distance = get_label_from_name(only_names[i])
        if int(label_distance) == int(test_label):# or check_if_same_category(test_label,label_distance) == 1:
            found = 1
            idx = i+1
            print('found')
            break
    return found,idx

# Compute correct matches

In [8]:
n = 1 #Top-1 match
correct = 0

for i in range(0,X_phish_new.shape[0]):
    distances_to_train = pairwise_distance[i,:]
    idx,values = find_min_distances(np.ravel(distances_to_train),n)
    names_min_distance,only_names,min_distances = find_names_min_distances(idx,values)
    found,found_idx = check_if_target_in_top(all_labels_phish_new[i,0],only_names)
    print('Closest: '+names_min_distance)
    
    if found == 1:
        correct += 1

print("Correct match percentage: " + str(correct/X_phish_new.shape[0]))

***
Target: 1.0
Closest: Phish: 050b0c0c0a.png
***
Target: 1.0
Closest: Phish: 050b0c0c0a.png
***
Target: 1.0
Closest: Legit: 050b0c0c0a.png
***
Target: 1.0
Closest: Phish: 694f02d813.png
***
Target: 1.0
Closest: Phish: 0bda4b979c.png
***
Target: 1.0
Closest: Legit: 050b0c0c0a.png
***
Target: 1.0
Closest: Legit: 0bda4b979c.png
***
Target: 1.0
Closest: Phish: 050b0c0c0a.png
***
Target: 1.0
Closest: Legit: 050b0c0c0a.png
***
Target: 1.0
Closest: Legit: 0bda4b979c.png
***
Target: 1.0
Closest: Phish: 0bda4b979c.png
***
Target: 1.0
Closest: Phish: 050b0c0c0a.png
***
Target: 1.0
Closest: Phish: 694f02d813.png
***
Target: 1.0
Closest: Legit: 050b0c0c0a.png
***
Target: 1.0
Closest: Phish: 050b0c0c0a.png
***
Target: 1.0
Closest: Legit: 9c7553e453.png
***
Target: 1.0
Closest: Phish: 0bda4b979c.png
***
Target: 1.0
Closest: Phish: 0bda4b979c.png
***
Target: 1.0
Closest: Legit: 050b0c0c0a.png
***
Target: 1.0
Closest: Phish: 0bda4b979c.png
***
Target: 1.0
Closest: Legit: 050b0c0c0a.png
***
Target: 1

***
Target: 17.0
Closest: Phish: ff2ce13f48.png
***
Target: 17.0
found
Closest: Legit: 5ac3121c96.png
***
Target: 17.0
Closest: Legit: 8ff337fa31.png
***
Target: 17.0
Closest: Phish: 5f5bc80d04.png
***
Target: 17.0
Closest: Legit: 8ff337fa31.png
***
Target: 17.0
found
Closest: Legit: 5ac3121c96.png
***
Target: 17.0
Closest: Legit: 8ff337fa31.png
***
Target: 17.0
Closest: Legit: c361548e91.png
***
Target: 17.0
found
Closest: Legit: 5ac3121c96.png
***
Target: 17.0
found
Closest: Legit: 5ac3121c96.png
***
Target: 17.0
Closest: Legit: 8ff337fa31.png
***
Target: 17.0
found
Closest: Legit: 5ac3121c96.png
***
Target: 17.0
found
Closest: Legit: 5ac3121c96.png
***
Target: 17.0
Closest: Legit: 6cd4e58c66.png
***
Target: 17.0
found
Closest: Legit: 5ac3121c96.png
***
Target: 17.0
Closest: Legit: eff9dfbccd.png
***
Target: 17.0
found
Closest: Legit: 5ac3121c96.png
***
Target: 17.0
found
Closest: Legit: 5ac3121c96.png
***
Target: 17.0
Closest: Legit: 1d6ab5ef56.png
***
Target: 17.0
found
Closest: Le